运行 20 次非对称谱估计器；

每轮提取实部在 [0.95,1.05] 区间内的绿色谱点（Im≈0, Abs≤R）；

若某轮中恰好存在 2 个符合条件的绿色点，将其与该轮中前 2 个红点一起用于评估；

最终对所有满足条件的轮次进行评估，返回平均对齐分数和标准差；

可选可视化热力图（如启用）。

In [6]:
import os
import random
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import kv as besselk
from scipy.stats import gaussian_kde
from scipy.linalg import qr
from numpy.linalg import norm
from itertools import combinations

# =================== 设置参数与设备 ===================
n, p, alpha, alpha_c = 5000, 2, 5, 0.59375
d = int(n / alpha)
R = np.sqrt(alpha / alpha_c)
thresh = alpha / alpha_c
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# =================== 子空间对齐函数 ===================
def alignment_score(W_hat, W_star_2d):
    Q1, _ = qr(W_hat, mode='economic')
    Q2, _ = qr(W_star_2d, mode='economic')
    overlap = Q1.T @ Q2
    _, s, _ = np.linalg.svd(overlap)
    return np.mean(s)

# =================== 主循环（多轮） ===================
num_trials = 20
qualified_trials = []
align_results = []

base_seeds = np.arange(0, 10000, step=500)[:num_trials]
noise = np.random.randint(0, 100, size=num_trials)
seeds = (base_seeds + noise).tolist()
print("使用的种子:", seeds)

for trial_id, seed in enumerate(seeds):
    torch.manual_seed(seed)
    np.random.seed(seed)

    # ----- 数据生成 -----
    W_star = torch.randn(d, p, device=device)
    X = torch.randn(n, d, device=device) / np.sqrt(d)
    Z = X @ W_star
    y = Z[:, 0] * Z[:, 1]

    y_np = y.cpu().numpy()
    abs_y = np.abs(y_np)
    K0, K1 = besselk(0, abs_y), besselk(1, abs_y)
    lambda_y = abs_y * (K1 / (K0 + 1e-6)) - 1
    lambda_y[np.isnan(lambda_y)] = 0

    G_y_np = np.zeros((n, p, p), dtype=np.float32)
    G_y_np[:, 0, 0] = lambda_y
    G_y_np[:, 1, 1] = lambda_y
    G_y_np[:, 0, 1] = y_np
    G_y_np[:, 1, 0] = y_np
    G_y = torch.tensor(G_y_np, device=device)

    XXT = X @ X.T
    XXT.fill_diagonal_(0.0)
    L_blocks = XXT[:, :, None, None] * G_y[None, :, :, :]
    L = L_blocks.permute(0, 2, 1, 3).reshape(n * p, n * p)

    eigvals, eigvecs = torch.linalg.eig(L)
    eigvals_np = eigvals.cpu().numpy()
    eigvecs_np = eigvecs.cpu().numpy()

    # ----- 红色点（前2大 Re） -----
    real_eigs = [z for z in eigvals_np if np.isclose(z.imag, 0.0, atol=1e-6)]
    red_vals = sorted(real_eigs, key=lambda z: z.real, reverse=True)[:2]

    # ----- 内部绿色点选取（Re ∈ [0.95, 1.05], |λ| ≤ R, Im≈0）-----
    green_zone = [
        z for z in real_eigs
        if 0.95 <= z.real <= 1.05 and np.abs(z) <= R and np.abs(z.imag) < 1e-6
    ]

    if len(green_zone) == 2:
        print(f"[✓] 第 {trial_id+1} 轮满足条件（2个绿色点）")
        qualified_trials.append(seed)

        # ----- 特征向量映射 -----
        eigval_to_label = {
            red_vals[0]: "Red1", red_vals[1]: "Red2",
            green_zone[0]: "Green1", green_zone[1]: "Green2"
        }
        label_order = list(eigval_to_label.values())
        eigval_list = list(eigval_to_label.keys())

        # ----- 构造 W_star_2d -----
        W_star_np = W_star.cpu().numpy()
        W_star_2d = np.vstack([W_star_np[:, i].reshape(-1,1) for i in range(p)])

        # ----- 单点对齐度 -----
        single_scores = []
        for val in eigval_list:
            idx = np.argmin(np.abs(eigvals_np - val))
            v = eigvecs_np[:, idx].reshape(n, p)
            v0 = X.T @ v[:, 0]
            v1 = X.T @ v[:, 1]
            mapped = np.vstack([v0.reshape(-1,1), v1.reshape(-1,1)])
            mapped /= (norm(mapped, axis=0, keepdims=True) + 1e-12)
            score = alignment_score(mapped, W_star_2d)
            single_scores.append(score)

        # ----- 双点组合对齐度 -----
        pair_scores = []
        for i in range(len(eigval_list)):
            for j in range(i+1, len(eigval_list)):
                idx1 = np.argmin(np.abs(eigvals_np - eigval_list[i]))
                idx2 = np.argmin(np.abs(eigvals_np - eigval_list[j]))
                v1 = eigvecs_np[:, idx1].reshape(n, p)
                v2 = eigvecs_np[:, idx2].reshape(n, p)
                mapped = np.vstack([
                    X.T @ v1[:, 0], X.T @ v1[:, 1],
                    X.T @ v2[:, 0], X.T @ v2[:, 1]
                ]).reshape(2*d, 2)
                mapped /= (norm(mapped, axis=0, keepdims=True) + 1e-12)
                score = alignment_score(mapped, W_star_2d)
                pair_scores.append(score)

        # ----- 记录平均对齐度 -----
        align_results.append({
            "seed": seed,
            "single_mean": np.mean(single_scores),
            "pair_mean": np.mean(pair_scores)
        })

# =================== 结果输出 ===================
qualified_count = len(align_results)
print(f"\n共有 {qualified_count} 次运行满足两个绿色点 ∈ [0.95, 1.05] 的条件")

if qualified_count > 0:
    df_result = pd.DataFrame(align_results)
    print("\n[单点对齐度统计]")
    print(df_result["single_mean"].describe())

    print("\n[双点组合对齐度统计]")
    print(df_result["pair_mean"].describe())
else:
    print("⚠️ 未找到符合条件的轮次，可能需要调整参数或扩大 trials 数")


Device: cpu
使用的种子: [16, 570, 1099, 1585, 2093, 2597, 3079, 3528, 4029, 4509, 5062, 5506, 6067, 6539, 7022, 7539, 8048, 8528, 9046, 9520]
[✓] 第 2 轮满足条件（2个绿色点）
[✓] 第 3 轮满足条件（2个绿色点）
[✓] 第 4 轮满足条件（2个绿色点）
[✓] 第 6 轮满足条件（2个绿色点）
[✓] 第 7 轮满足条件（2个绿色点）
[✓] 第 9 轮满足条件（2个绿色点）
[✓] 第 11 轮满足条件（2个绿色点）
[✓] 第 13 轮满足条件（2个绿色点）
[✓] 第 18 轮满足条件（2个绿色点）

共有 9 次运行满足两个绿色点 ∈ [0.95, 1.05] 的条件

[单点对齐度统计]
count    9.000000
mean     0.502435
std      0.005177
min      0.497444
25%      0.498423
50%      0.500645
75%      0.503939
max      0.513597
Name: single_mean, dtype: float64

[双点组合对齐度统计]
count    9.000000
mean     0.031439
std      0.009253
min      0.014741
25%      0.025267
50%      0.032548
75%      0.037816
max      0.044211
Name: pair_mean, dtype: float64


In [20]:
import os
import random
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import kv as besselk
from scipy.stats import gaussian_kde
from scipy.linalg import qr
from numpy.linalg import norm
from itertools import combinations

# =================== 设置参数与设备 ===================
n, p, alpha, alpha_c = 5000, 2, 8, 0.59375
d = int(n / alpha)
R = np.sqrt(alpha / alpha_c)
thresh = alpha / alpha_c
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# =================== 子空间对齐函数 ===================
def alignment_score(W_hat, W_star_2d):
    Q1, _ = qr(W_hat, mode='economic')
    Q2, _ = qr(W_star_2d, mode='economic')
    overlap = Q1.T @ Q2
    _, s, _ = np.linalg.svd(overlap)
    return np.mean(s)

# =================== 主循环（多轮） ===================
num_trials = 20
qualified_trials = []
align_results = []

base_seeds = np.arange(0, 10000, step=500)[:num_trials]
noise = np.random.randint(0, 100, size=num_trials)
seeds = (base_seeds + noise).tolist()
print("使用的种子:", seeds)

for trial_id, seed in enumerate(seeds):
    torch.manual_seed(seed)
    np.random.seed(seed)

    # ----- 数据生成 -----
    W_star = torch.randn(d, p, device=device)
    X = torch.randn(n, d, device=device) / np.sqrt(d)
    Z = X @ W_star
    y = Z[:, 0] * Z[:, 1]

    y_np = y.cpu().numpy()
    abs_y = np.abs(y_np)
    K0, K1 = besselk(0, abs_y), besselk(1, abs_y)
    lambda_y = abs_y * (K1 / (K0 + 1e-6)) - 1
    lambda_y[np.isnan(lambda_y)] = 0

    G_y_np = np.zeros((n, p, p), dtype=np.float32)
    G_y_np[:, 0, 0] = lambda_y
    G_y_np[:, 1, 1] = lambda_y
    G_y_np[:, 0, 1] = y_np
    G_y_np[:, 1, 0] = y_np
    G_y = torch.tensor(G_y_np, device=device)

    XXT = X @ X.T
    XXT.fill_diagonal_(0.0)
    L_blocks = XXT[:, :, None, None] * G_y[None, :, :, :]
    L = L_blocks.permute(0, 2, 1, 3).reshape(n * p, n * p)

    eigvals, eigvecs = torch.linalg.eig(L)
    eigvals_np = eigvals.cpu().numpy()
    eigvecs_np = eigvecs.cpu().numpy()

    # ----- 红色点（前2大 Re） -----
    real_eigs = [z for z in eigvals_np if np.isclose(z.imag, 0.0, atol=1e-6)]
    red_vals = sorted(real_eigs, key=lambda z: z.real, reverse=True)[:2]

    # ----- 内部绿色点选取（Re ∈ [0.95, 1.05], |λ| ≤ R, Im≈0）-----
    green_zone = [
        z for z in real_eigs
        if 0.95 <= z.real <= 1.05 and np.abs(z) <= R and np.abs(z.imag) < 1e-6
    ]

    if len(green_zone) == 2:
        print(f"[✓] 第 {trial_id+1} 轮满足条件（2个绿色点）")
        qualified_trials.append(seed)

        # ----- 特征向量映射 -----
        eigval_to_label = {
            red_vals[0]: "Red1", red_vals[1]: "Red2",
            green_zone[0]: "Green1", green_zone[1]: "Green2"
        }

        # ----- 构造 W_star_2d -----
        W_star_np = W_star.cpu().numpy()
        W_star_2d = np.vstack([W_star_np[:, i].reshape(-1,1) for i in range(p)])

        # ----- 单点对齐度 -----
        single_scores_dict = {}
        for eigval, label in eigval_to_label.items():
            idx = np.argmin(np.abs(eigvals_np - eigval))
            v = eigvecs_np[:, idx].reshape(n, p)
            v0 = X.T @ v[:, 0]
            v1 = X.T @ v[:, 1]
            mapped = np.vstack([v0.reshape(-1,1), v1.reshape(-1,1)])
            mapped /= (norm(mapped, axis=0, keepdims=True) + 1e-12)
            score = alignment_score(mapped, W_star_2d)
            single_scores_dict[label] = score

        # ----- 双点组合对齐度 -----
        pair_scores = []
        eigval_list = list(eigval_to_label.keys())
        for i in range(len(eigval_list)):
            for j in range(i+1, len(eigval_list)):
                idx1 = np.argmin(np.abs(eigvals_np - eigval_list[i]))
                idx2 = np.argmin(np.abs(eigvals_np - eigval_list[j]))
                v1 = eigvecs_np[:, idx1].reshape(n, p)
                v2 = eigvecs_np[:, idx2].reshape(n, p)
                mapped = np.vstack([
                    X.T @ v1[:, 0], X.T @ v1[:, 1],
                    X.T @ v2[:, 0], X.T @ v2[:, 1]
                ]).reshape(2*d, 2)
                mapped /= (norm(mapped, axis=0, keepdims=True) + 1e-12)
                score = alignment_score(mapped, W_star_2d)
                pair_scores.append(score)

        # ----- 记录结果 -----
        align_results.append({
            "seed": seed,
            **single_scores_dict,
            "pair_mean": np.mean(pair_scores)
        })

# =================== 结果输出 ===================
qualified_count = len(align_results)
print(f"\n共有 {qualified_count} 次运行满足两个绿色点 ∈ [0.95, 1.05] 的条件")

if qualified_count > 0:
    df_result = pd.DataFrame(align_results)
    print("\n[单点对齐度统计]")
    print(df_result[["Red1", "Red2", "Green1", "Green2"]].describe())

    print("\n[双点组合对齐度统计]")
    print(df_result["pair_mean"].describe())
else:
    print("⚠️ 未找到符合条件的轮次，可能需要调整参数或扩大 trials 数")


Device: cpu
使用的种子: [6, 595, 1044, 1592, 2062, 2506, 3008, 3572, 4055, 4572, 5067, 5520, 6024, 6587, 7058, 7548, 8014, 8536, 9074, 9519]
[✓] 第 1 轮满足条件（2个绿色点）
[✓] 第 4 轮满足条件（2个绿色点）
[✓] 第 6 轮满足条件（2个绿色点）
[✓] 第 9 轮满足条件（2个绿色点）
[✓] 第 11 轮满足条件（2个绿色点）
[✓] 第 13 轮满足条件（2个绿色点）
[✓] 第 14 轮满足条件（2个绿色点）
[✓] 第 16 轮满足条件（2个绿色点）

共有 8 次运行满足两个绿色点 ∈ [0.95, 1.05] 的条件

[单点对齐度统计]
           Red1      Red2    Green1    Green2
count  8.000000  8.000000  8.000000  8.000000
mean   0.526195  0.516981  0.605567  0.622585
std    0.010150  0.008665  0.008572  0.009957
min    0.508058  0.499888  0.591486  0.611415
25%    0.521928  0.514634  0.602569  0.614452
50%    0.529186  0.517036  0.603350  0.621552
75%    0.534044  0.522843  0.609820  0.629188
max    0.535644  0.527975  0.618039  0.637500

[双点组合对齐度统计]
count    8.000000
mean     0.037265
std      0.013855
min      0.016458
25%      0.029100
50%      0.037816
75%      0.044420
max      0.061045
Name: pair_mean, dtype: float64


In [43]:
import os
import random
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import kv as besselk
from scipy.stats import gaussian_kde
from scipy.linalg import qr
from numpy.linalg import norm
from itertools import combinations

# =================== 设置参数与设备 ===================
n, p, alpha, alpha_c = 5000, 2, 4.1, 0.59375
d = int(n / alpha)
R = np.sqrt(alpha / alpha_c)
thresh = alpha / alpha_c
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# =================== 子空间对齐函数 ===================
def alignment_score(W_hat, W_star_2d):
    Q1, _ = qr(W_hat, mode='economic')
    Q2, _ = qr(W_star_2d, mode='economic')
    overlap = Q1.T @ Q2
    _, s, _ = np.linalg.svd(overlap)
    return np.mean(s)

# =================== 主循环（多轮） ===================
num_trials = 20
qualified_trials = []
align_results = []

base_seeds = np.arange(0, 10000, step=500)[:num_trials]
noise = np.random.randint(0, 100, size=num_trials)
seeds = (base_seeds + noise).tolist()
print("使用的种子:", seeds)

for trial_id, seed in enumerate(seeds):
    torch.manual_seed(seed)
    np.random.seed(seed)

    # ----- 数据生成 -----
    W_star = torch.randn(d, p, device=device)
    X = torch.randn(n, d, device=device) / np.sqrt(d)
    Z = X @ W_star
    y = Z[:, 0] * Z[:, 1]

    y_np = y.cpu().numpy()
    abs_y = np.abs(y_np)
    K0, K1 = besselk(0, abs_y), besselk(1, abs_y)
    lambda_y = abs_y * (K1 / (K0 + 1e-6)) - 1
    lambda_y[np.isnan(lambda_y)] = 0

    G_y_np = np.zeros((n, p, p), dtype=np.float32)
    G_y_np[:, 0, 0] = lambda_y
    G_y_np[:, 1, 1] = lambda_y
    G_y_np[:, 0, 1] = y_np
    G_y_np[:, 1, 0] = y_np
    G_y = torch.tensor(G_y_np, device=device)

    XXT = X @ X.T
    XXT.fill_diagonal_(0.0)
    L_blocks = XXT[:, :, None, None] * G_y[None, :, :, :]
    L = L_blocks.permute(0, 2, 1, 3).reshape(n * p, n * p)

    eigvals, eigvecs = torch.linalg.eig(L)
    eigvals_np = eigvals.cpu().numpy()
    eigvecs_np = eigvecs.cpu().numpy()

    # ----- 红色点（前2大 Re） -----
    real_eigs = [z for z in eigvals_np if np.isclose(z.imag, 0.0, atol=1e-6)]
    red_vals = sorted(real_eigs, key=lambda z: z.real, reverse=True)[:2]

    # ----- 内部绿色点选取（Re ∈ [0.95, 1.05], |λ| ≤ R, Im≈0）-----
    # 跳脱阈值定义，可设为谱堆积中心 + margin（例如 0.95）
    jump_thresh = 0.95  # 你可以根据经验或 KDE 调整

# 所有实数谱点中筛选出满足条件的绿色候选点
    green_candidates = [
        z for z in real_eigs
        if z.real > jump_thresh and np.abs(z) < R and np.abs(z.imag) < 1e-6
    ]

# 仅保留实部最大的前两个点
    green_zone = sorted(green_candidates, key=lambda z: z.real, reverse=True)[:2]



    if len(green_zone) == 2:
        print(f"[✓] 第 {trial_id+1} 轮满足条件（2个绿色点）")
        qualified_trials.append(seed)

        # ----- 特征向量映射 -----
        eigval_to_label = {
            red_vals[0]: "Red1", red_vals[1]: "Red2",
            green_zone[0]: "Green1", green_zone[1]: "Green2"
        }

        # ----- 构造 W_star_2d -----
        W_star_np = W_star.cpu().numpy()
        W_star_2d = np.vstack([W_star_np[:, i].reshape(-1,1) for i in range(p)])

        # ----- 单点对齐度 -----
        single_scores_dict = {}
        for eigval, label in eigval_to_label.items():
            idx = np.argmin(np.abs(eigvals_np - eigval))
            v = eigvecs_np[:, idx].reshape(n, p)
            v0 = X.T @ v[:, 0]
            v1 = X.T @ v[:, 1]
            mapped = np.vstack([v0.reshape(-1,1), v1.reshape(-1,1)])
            mapped /= (norm(mapped, axis=0, keepdims=True) + 1e-12)
            score = alignment_score(mapped, W_star_2d)
            single_scores_dict[label] = score

        # ----- 双点组合对齐度 -----
        pair_scores = []
        eigval_list = list(eigval_to_label.keys())
        for i in range(len(eigval_list)):
            for j in range(i+1, len(eigval_list)):
                idx1 = np.argmin(np.abs(eigvals_np - eigval_list[i]))
                idx2 = np.argmin(np.abs(eigvals_np - eigval_list[j]))
                v1 = eigvecs_np[:, idx1].reshape(n, p)
                v2 = eigvecs_np[:, idx2].reshape(n, p)
                mapped = np.vstack([
                    X.T @ v1[:, 0], X.T @ v1[:, 1],
                    X.T @ v2[:, 0], X.T @ v2[:, 1]
                ]).reshape(2*d, 2)
                mapped /= (norm(mapped, axis=0, keepdims=True) + 1e-12)
                score = alignment_score(mapped, W_star_2d)
                pair_scores.append(score)

        # ----- 记录结果 -----
        align_results.append({
            "seed": seed,
            **single_scores_dict,
            "pair_mean": np.mean(pair_scores)
        })

# =================== 结果输出 ===================
qualified_count = len(align_results)
print(f"\n共有 {qualified_count} 次运行满足两个绿色点 ∈ [0.95, 1.05] 的条件")

if qualified_count > 0:
    df_result = pd.DataFrame(align_results)
    print("\n[单点对齐度统计]")
    print(df_result[["Red1", "Red2", "Green1", "Green2"]].describe())

    print("\n[双点组合对齐度统计]")
    print(df_result["pair_mean"].describe())
else:
    print("⚠️ 未找到符合条件的轮次，可能需要调整参数或扩大 trials 数")


Device: cpu
使用的种子: [98, 562, 1078, 1564, 2088, 2542, 3005, 3524, 4027, 4552, 5072, 5546, 6096, 6584, 7061, 7598, 8092, 8599, 9068, 9586]
[✓] 第 1 轮满足条件（2个绿色点）
[✓] 第 2 轮满足条件（2个绿色点）
[✓] 第 3 轮满足条件（2个绿色点）
[✓] 第 4 轮满足条件（2个绿色点）
[✓] 第 5 轮满足条件（2个绿色点）
[✓] 第 6 轮满足条件（2个绿色点）
[✓] 第 7 轮满足条件（2个绿色点）
[✓] 第 8 轮满足条件（2个绿色点）
[✓] 第 9 轮满足条件（2个绿色点）
[✓] 第 10 轮满足条件（2个绿色点）
[✓] 第 11 轮满足条件（2个绿色点）
[✓] 第 12 轮满足条件（2个绿色点）
[✓] 第 13 轮满足条件（2个绿色点）
[✓] 第 14 轮满足条件（2个绿色点）
[✓] 第 15 轮满足条件（2个绿色点）
[✓] 第 16 轮满足条件（2个绿色点）
[✓] 第 17 轮满足条件（2个绿色点）
[✓] 第 18 轮满足条件（2个绿色点）
[✓] 第 19 轮满足条件（2个绿色点）
[✓] 第 20 轮满足条件（2个绿色点）

共有 20 次运行满足两个绿色点 ∈ [0.95, 1.05] 的条件

[单点对齐度统计]
            Red1       Red2     Green1     Green2
count  20.000000  20.000000  20.000000  20.000000
mean    0.428104   0.416209   0.235169   0.291370
std     0.010155   0.012194   0.252163   0.259175
min     0.406993   0.387574   0.002064   0.000784
25%     0.422988   0.408193   0.010062   0.014498
50%     0.429776   0.415983   0.027668   0.506180
75%     0.434849   0.425932   0.50